In [1]:
!pip install shapely


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.0 MB/s  0:00:00


In [2]:
import json
import pickle
import itertools
import numpy as np
import pandas as pd
import helpers as lib
from rinet.metrics import bhattacharyya_gaussian_distance
from sklearn.metrics import precision_recall_fscore_support
from scipy.spatial import ConvexHull

pred_path = '../benchmarking/predictions/rinet_v2_liver_2d_ornone.pkl'
data_path = '../../data/liver/'
ri_sources = ['abim', 'leeds']
ris_path = {}
for source in ri_sources:
    ris_path[source] = f"{data_path}/established_ri/{source}_ris.json"


def load_p_y(pair, gender):
    """Load predicted and directly estimated reference regions"""
    
    # filter to given analyte pair and gender
    idx = metadata[(metadata['analyte_pair'] == str(pair)) & (metadata['gender'] == gender)].index.to_numpy()[0]
    p = predictions[idx]
    y = targets[idx]

    # convert from dictionary
    p = [p['mean'], p['covariance'], p['reference_fraction']]

    # get region vertices
    region_p = lib.get_ellipse_vertices(p[0], p[1])
    region_y = lib.get_ellipse_vertices(y[0], y[1])

    return region_p, region_y, p, y
    


In [3]:
# load data
predictions = pickle.load(open(pred_path, 'rb'))[1]
targets = pickle.load(open(f"{data_path}/outlier_removal_none/2d/targets.pkl", 'rb'))
metadata = pd.read_csv(f"{data_path}/outlier_removal_none/2d/metadata.csv", index_col=0)

df = pd.read_csv(f"{data_path}/liver_preprocessed.csv")
analytes = df.columns[3:]  # list of analytes
analytes = analytes.drop(['cholesterol', 'cholinesterase', 'alkaline phosphatase'])  # remove 3 analytes
log_analytes = list(analytes)  # transform all analytes


In [9]:
# run prediction
unique_pairs = sorted(list(set(itertools.combinations(analytes, 2))))  # get all pairs of analytes
results = []
for gender in ['M', 'F']:  # loop genders
    
    print(gender)
    
    for pair in unique_pairs:  # loop pairs
        
        print('\t', pair)
        pair = list(pair)

        # get estimated regions
        region_p, region_y, gaus_p, gaus_y = load_p_y(tuple(pair), gender)
        
        # get established RIs
        region_ri = {}
        for source in ri_sources:
            ris_dict = json.load(open(ris_path[source], 'r'))  # ris
            region_ri[source] = lib.get_ri_vertices(ris_dict, pair, gender)

        # inverse transform regions
        for c in range(2):
            if pair[c] in log_analytes:
                region_p[:, c] = np.exp(region_p[:, c])
                region_y[:, c] = np.exp(region_y[:, c])
            
        # get full dataset not transformed
        # outlier removal?
        data, labels = lib.get_data(
            df, pair, gender, log_analytes,
            outlier_removal=False,
            transform=False
        )

        ref_frac = len([i for i in labels if i == 'reference' ])/len(labels)
        
        # bhattacharyya coefficient
        bd = bhattacharyya_gaussian_distance(
            gaus_p[0], gaus_p[1], 
            gaus_y[0], gaus_y[1]
        )
        bc = np.exp(-bd)
        
        # get IoU between directly estimated and predicted regions
        iou_model = lib.iou(region_p, region_y)
        iou_ri = {}
        for source in ri_sources:
            iou_ri[source] = lib.iou(region_ri[source], region_y)        
        
        # estimate precision recall fscore and support of predicted positive/negative patients vs labels
        labels_nn = lib.predict_labels(data, region_p)
        assert len(labels_nn) == len(data), 'Error: number of labels != len dataset (model)'
        labels_ri = {}
        for source in ri_sources:
            labels_ri[source] = lib.predict_labels(data, region_ri[source])
            assert len(labels_ri[source]) == len(data), 'Error: number of labels != len dataset (RIs)'
        labels_direct = lib.predict_labels(data, region_y)
        assert len(labels_direct) == len(data), 'Error: number of labels != len dataset (direct estimate)'
        
        # run evaluation only using confirmed normal and HCV (not just "abnormal")
        idx = np.array([i != 'abnormal' for i in labels])
        labels_target = labels[idx]
        labels_target = labels_target != 'reference'
        scores_nn = precision_recall_fscore_support(labels_target, labels_nn[idx])
        scores_ri = {}
        for source in ri_sources:
            scores_ri[source] = precision_recall_fscore_support(labels_target, labels_ri[source][idx])
        scores_direct = precision_recall_fscore_support(labels_target, labels_direct[idx])
        
        # get standardized vertices for standardized areas
        region_p_standard = (region_p-data.mean(axis=0))/data.std(axis=0)
        region_ri_standard = {}
        for source in ri_sources:
            region_ri_standard[source] = (region_ri[source]-data.mean(axis=0))/data.std(axis=0)
        region_y_standard = (region_y-data.mean(axis=0))/data.std(axis=0)
        
        # compile results
        result = {
            'pair': pair,
            'gender': gender,
            'labels': labels,
            'model': {
                'volume': ConvexHull(region_p).volume,
                'volume_standard': ConvexHull(region_p_standard).volume,
                'coverage': len(np.where(~labels_nn[idx] & ~labels_target)[0])/len(np.where(~labels_target)[0]),
                'bhattacharyya_coefficient': bc,
                'iou': iou_model,
                'vertices': region_p,
                'precision': scores_nn[0],
                'recall': scores_nn[1],
                'f1score': scores_nn[2],
                'support': scores_nn[3],
                'reference_frac': gaus_p[2],
                'reference_frac_error': np.abs(gaus_p[2] - ref_frac),
                'labels': labels_nn,
                'model': gaus_p
            },
            'direct': {
                'volume': ConvexHull(region_y).volume,
                'volume_standard': ConvexHull(region_y_standard).volume,
                'coverage': len(np.where(~labels_direct[idx] & ~labels_target)[0])/len(np.where(~labels_target)[0]),
                'bhattacharyya_coefficient': 1,
                'iou': 1,
                'vertices': region_y,
                'precision': scores_direct[0],
                'recall': scores_direct[1],
                'f1score': scores_direct[2],
                'support': scores_direct[3],
                'labels': labels_direct,
                'model': gaus_y
            }
        }
        result['model']['coverage_area_ratio'] = result['model']['coverage'] / result['model']['volume_standard']
        result['direct']['coverage_area_ratio'] = result['direct']['coverage'] / result['model']['volume_standard']
        
        for source in ri_sources:
            result['ri_'+source] = {
                'volume': ConvexHull(region_ri[source]).volume,
                'volume_standard': ConvexHull(region_ri_standard[source]).volume,
                'coverage': len(np.where(~labels_ri[source][idx] & ~labels_target)[0])/len(np.where(~labels_target)[0]),
                'bhattacharyya_coefficient': None,
                'iou': iou_ri[source],
                'vertices': region_ri[source],
                'precision': scores_ri[source][0],
                'recall': scores_ri[source][1],
                'f1score': scores_ri[source][2],
                'support': scores_ri[source][3],
                'labels': labels_ri[source],
                'model': None
            }
            result['ri_'+source]['coverage_area_ratio'] = result['ri_'+source]['coverage'] / result['ri_'+source]['volume_standard']

        results.append(result)
        

M
	 ('alanine aminotransferase', 'albumin')
	 ('alanine aminotransferase', 'aspartate aminotransferase')
	 ('alanine aminotransferase', 'bilirubin')
	 ('alanine aminotransferase', 'creatinine')
	 ('alanine aminotransferase', 'gamma-glutamyl transferase')
	 ('alanine aminotransferase', 'total protein')
	 ('albumin', 'aspartate aminotransferase')
	 ('albumin', 'bilirubin')
	 ('albumin', 'creatinine')
	 ('albumin', 'gamma-glutamyl transferase')
	 ('albumin', 'total protein')
	 ('aspartate aminotransferase', 'bilirubin')
	 ('aspartate aminotransferase', 'creatinine')
	 ('aspartate aminotransferase', 'gamma-glutamyl transferase')
	 ('aspartate aminotransferase', 'total protein')
	 ('bilirubin', 'creatinine')
	 ('bilirubin', 'gamma-glutamyl transferase')
	 ('bilirubin', 'total protein')
	 ('creatinine', 'gamma-glutamyl transferase')
	 ('creatinine', 'total protein')
	 ('gamma-glutamyl transferase', 'total protein')
F
	 ('alanine aminotransferase', 'albumin')
	 ('alanine aminotransferase', 'a

In [10]:
# can also check this is the same as recall[0]
print('Average Coverage Model: '+str(np.mean([i['model']['coverage'] for i in results])))
for source in ri_sources:
    print(f"Average Coverage RI ({source}): {str(np.mean([i['ri_'+source]['coverage'] for i in results]))}")
print('Average Coverage Direct: '+str(np.mean([i['direct']['coverage'] for i in results])))
print('\n')

print('Average Standard Area Model: '+str(np.mean([i['model']['volume_standard'] for i in results])))
for source in ri_sources:
    print(f"Average Standard Area RI ({source}): {str(np.mean([i['ri_'+source]['volume_standard'] for i in results]))}")
print('Average Standard Area Direct: '+str(np.mean([i['direct']['volume_standard'] for i in results])))
print('\n')

print('Average C-A-Ratio Model: '+str(np.mean([i['model']['coverage_area_ratio'] for i in results])))
for source in ri_sources:
    print(f"Average C-A-Ratio RI ({source}): {str(np.mean([i['ri_'+source]['coverage_area_ratio'] for i in results]))}")
print('Average C-A-Ratio Direct: '+str(np.mean([i['direct']['coverage_area_ratio'] for i in results])))
print('\n')

print('Average Sensitivity Model: '+str(np.mean([i['model']['recall'][1] for i in results])))
for source in ri_sources:
    print(f"Average Sensitivity RI ({source}): {str(np.mean([i['ri_'+source]['recall'][1] for i in results]))}")
print('Average Sensitivity Direct: '+str(np.mean([i['direct']['recall'][1] for i in results])))
print('\n')

print('Average IoU Model: '+str(np.mean([i['model']['iou'] for i in results])))
for source in ri_sources:
    print(f"Average IOU RI ({source}): {str(np.mean([i['ri_'+source]['iou'] for i in results]))}")
print('Average BC Model: '+str(np.mean([i['model']['bhattacharyya_coefficient'] for i in results])))
print('Average Ref Frac Model: '+str(np.mean([i['model']['reference_frac'] for i in results])))
print('Average Ref Frac Error Model: '+str(np.mean([i['model']['reference_frac_error'] for i in results])))



Average Coverage Model: 0.9379361514398011
Average Coverage RI (abim): 0.8231588536593749
Average Coverage RI (leeds): 0.9157270432035812
Average Coverage Direct: 0.9524544798267426


Average Standard Area Model: 2.659774652190825
Average Standard Area RI (abim): 4.0462364352722044
Average Standard Area RI (leeds): 3.2604401730613852
Average Standard Area Direct: 2.6904340999865544


Average C-A-Ratio Model: 0.5725806010522303
Average C-A-Ratio RI (abim): 0.532090291137621
Average C-A-Ratio RI (leeds): 0.4175499468324306
Average C-A-Ratio Direct: 0.5831257371590538


Average Sensitivity Model: 0.6664502164502165
Average Sensitivity RI (abim): 0.6841125541125541
Average Sensitivity RI (leeds): 0.6066666666666667
Average Sensitivity Direct: 0.6660173160173161


Average IoU Model: 0.8289675972892687
Average IOU RI (abim): 0.486620509628178
Average IOU RI (leeds): 0.5353433770247437
Average BC Model: 0.9908830746853072
Average Ref Frac Model: 0.7010845
Average Ref Frac Error Model: 0.06129

In [11]:
print('Std. Dev. Ref Frac Model: '+str(np.std([i['model']['reference_frac'] for i in results])))


Std. Dev. Ref Frac Model: 0.057553567


In [12]:
lib.save_pickle(results, './results.pkl')
